In [1]:
# Dataset
!gdown 
!unzip -q Celeb-df-V2-faces-extracted.zip

# Model
!gdown 

Downloading...
From (original): https://drive.google.com/uc?id=1O3m1GXyS4OpHk5gkBSCJauRPTcw8y-n_
From (redirected): https://drive.google.com/uc?id=1O3m1GXyS4OpHk5gkBSCJauRPTcw8y-n_&confirm=t&uuid=d2dc27a7-b18b-4562-ae20-95da44e7a0ae
To: /content/Celeb-df-V2-faces-extracted.zip
100% 260M/260M [00:06<00:00, 39.8MB/s]


In [21]:
import tensorflow as tf
import os
import cv2
import numpy as np
import shutil

In [4]:
model = tf.keras.models.load_model('/content/cnn_fully.keras')

In [14]:
def read_video(video_path):
    video_frames = []
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, current_frame = cap.read()
        if not ret: break
        current_frame = cv2.resize(current_frame, (299, 299))
        current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2RGB)
        video_frames.append(current_frame)
    cap.release()

    video_frames = np.array([video_frames]) / 255.

    return video_frames


real_videos = os.listdir("/content/Celeb-df-V2-faces-extracted/test/real")
fake_videos = os.listdir("/content/Celeb-df-V2-faces-extracted/test/fake")

real_videos_path = [(os.path.join("/content/Celeb-df-V2-faces-extracted/test/real", video_path), np.array([1., 0.])) for video_path in real_videos]
fake_videos_path = [(os.path.join("/content/Celeb-df-V2-faces-extracted/test/fake", video_path), np.array([0., 1.])) for video_path in fake_videos]

dataset = real_videos_path + fake_videos_path

In [19]:
wrong_videos_path = []

for i, (video_path, label) in enumerate(dataset):
    video = read_video(video_path)
    prediction = model.predict(video, verbose = 0)

    if np.argmax(prediction) != np.argmax(label):
        wrong_videos_path.append(video_path)

    if (i+1) % 50 == 0:
        print(f"{i+1}", end = " | ")

total_videos = len(dataset)
wrong = len(wrong_videos_path)
right = total_videos - wrong

print("Total videos:", total_videos)
print("Correct Videos:", right)
print("Wrong videos:", wrong)

49
99
149
199
249
299
349
399
449
499
Total videos: 518
Correct Videos: 504
Wrong videos: 14


In [26]:
dirpath = "wrong_classification"
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

os.makedirs(dirpath + "/real")
os.makedirs(dirpath + "/fake")

for path in wrong_videos_path:
    if "real" in path:
        shutil.copy(src = path, dst = "wrong_classification/real")
    else:
        shutil.copy(src = path, dst = "wrong_classification/fake")

!zip -r wrong_classification.zip wrong_classification

  adding: wrong_classification/ (stored 0%)
  adding: wrong_classification/real/ (stored 0%)
  adding: wrong_classification/real/id32_0006_segment_1.mp4 (deflated 29%)
  adding: wrong_classification/real/id31_0007_segment_1.mp4 (deflated 3%)
  adding: wrong_classification/real/id21_0009_segment_1.mp4 (deflated 5%)
  adding: wrong_classification/real/00279_segment_1.mp4 (deflated 19%)
  adding: wrong_classification/real/id35_0003_segment_1.mp4 (deflated 3%)
  adding: wrong_classification/real/id6_0005_segment_1.mp4 (deflated 8%)
  adding: wrong_classification/real/id53_0003_segment_1.mp4 (deflated 4%)
  adding: wrong_classification/fake/ (stored 0%)
  adding: wrong_classification/fake/id29_id30_0009_segment_1.mp4 (deflated 10%)
  adding: wrong_classification/fake/id35_id16_0007_segment_1.mp4 (deflated 16%)
  adding: wrong_classification/fake/id10_id12_0004_segment_1.mp4 (deflated 10%)
  adding: wrong_classification/fake/id17_id1_0000_segment_1.mp4 (deflated 7%)
  adding: wrong_classific